# Search Parameters

In [ ]:
seed = 70 # seed for NMF topic model
num_topics = 12
labels = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
vis_seed = 6 # seed for t-SNE visualization
vis_angle = 135 # rotation angle for visualization

# Import Modules

In [ ]:
import requests
import gensim
from gensim.utils import simple_preprocess
import numpy as np
import re
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.decomposition
import sklearn.feature_extraction
from wordcloud import WordCloud
from IPython.display import display
from collections import defaultdict

import seaborn as sns
sns.set('paper')

import logging
logging.getLogger().setLevel(logging.INFO)

# some python 3 trickery
import sys
if sys.version_info[0] >= 3:
    unicode = str


# Useful Functions

In [ ]:
def prepare_fig(w=1, h=None):
    if h is None: h = w
    figsize = (6 * w, 3 * h)
    sns.set(rc={'figure.figsize': figsize})
    fig = plt.figure(figsize=figsize)
    plt.clf()
    return fig

def top_k(mapping, k=10):
    return sorted(mapping.keys(), key=lambda x: mapping[x])[::-1][:k]

pd.set_option('display.max_rows', 250)

def plot_statistic(fun):
    count = defaultdict(int)

    for row in fun:
        if row:
            count[unicode(row)] += 1

    top_keys = top_k(count, 50)

    prepare_fig(1, 4)
    plt.xlabel("No. publications")
    plt.barh(
        range(len(top_keys)),
        [count[a] for a in top_keys])
    plt.yticks(
        range(len(top_keys)), 
        [key[:50] for key in top_keys])
    plt.show()

# Perform Query

In [ ]:
# Some query examples
# "protein AND (molecular dynamics) AND year_published:[2000 TO 2018]"
# "author.display_name:(Jingjie AND Yeo) AND year_published:[2000 TO 2018]"
# "(autonomous driving) OR (self-driving car) OR (robotic car)"
# "(silk) OR (collagen) AND (biomaterial) AND (molecular dynamics)"

url = 'https://api.lens.org/scholarly/search'

data = '''{
    "query": {
        "query_string": {
            "query": "(community participatory research) AND year_published:[2017 TO 2018]",
            "default_operator": "and"
        }
    },
    "size": 500,
    "sort": [
            {
            "year_published": "desc"
            }
     ]
}'''

# Use your LENS API key here
headers = {'Authorization': 'Bearer YOUR_API_KEY', 'Content-Type': 'application/json'}

response = requests.post(url, data=data, headers=headers)
if response.status_code != requests.codes.ok:
  print(response.status_code)
else:
  print(response.text)

j = response.json()

# Raw Data
df_raw = pd.DataFrame.from_dict(j['data'])
df_raw = df_raw.reset_index() # make sure indexes pair with number of rows

# Filter journal articles only
df = df_raw[df_raw['publication_type'].str.contains('journal article', na=False)] 
df = df.reset_index() # make sure indexes pair with number of rows

In [ ]:
# Start processing
texts = []
for index, row in df.iterrows():
    text = (row['title'] or '') + ' ' 
    
    # Skip over empty results
    if not pd.isna(row['abstract']):
        text = text + (row['abstract'] or '')
    text = text.lower()
    text = re.sub('[^a-zA-Z0-9]', ' ', text) # Replace punctation by spaces
    texts.append([w for w in text.split(' ') if w]) # Split on spaces, remove empty items

# Analysis

In [ ]:
# Publications per year
year_count = defaultdict(int)

for index, row in df.iterrows():
    # Skip over empty results
    if not pd.isna(row['year_published']):
        year_count[int(row['year_published'])] += 1
     
years = range(2009, 2023)

prepare_fig(1.8, 1.8)
plt.ylabel("No. publications",fontsize=18)
plt.xlabel("Year",fontsize=18)
plt.bar(
    years,
    [year_count[y] for y in years])
plt.xticks(years);
plt.tick_params(labelsize=18)
plt.tight_layout()
plt.savefig('years_ad.pdf')

In [ ]:
# Publications per type
plot_statistic(df_raw['publication_type'])

In [ ]:
# Publications per author
author = []
for a in df['authors']:
    for b in a:
        try:
            author.append(b['last_name'] + ' ' + b['initials'])
        except Exception:
            continue

plot_statistic(author)

In [ ]:
# Publications per institute
def clean_affiliation(name):
    name = unicode(name).title()
    pairs = [
        ['University', 'U'],
        ['Universitat', 'U'],
        ['Laboratories', 'Lab'],
        ['Laboratory', 'Lab'],
        ['National', 'Nat'],
        ['Corporation', 'Corp'],
        ['Technology', 'Tech'],
        ['Institute', 'Inst'],
        ['And', '&'],
        ['Education', 'Ed'],
        ['Engineering', 'Eng'],
        ['Mechanical', 'Mech'],
    ]
    
    for needle, replacement in pairs:
        name = name.replace(needle, replacement)
    return name

inst = []
for a in df['authors']:
    for b in a:
        
        try:
            b['affiliations']
        except KeyError:
            continue
        
        for c in b['affiliations']:
            affil = clean_affiliation(c['name'].split(',')[0])
            inst.append(affil)
            
plot_statistic(inst)

In [ ]:
# Publications per publication source, conference/journal
def clean_journal(name):
    name = unicode(name).title()
    pairs = [
        ['Journal', 'J'],
        ['Nature', 'Nat'],
        ['Communications', 'Comm'],
        ['Materials', 'Mat'],
        ['Chemistry', 'Chem'],
        ['Advanced', 'Adv'],
        ['Technology', 'Tech'],
        ['Institute', 'Inst'],
        ['And', '&'],
        ['Education', 'Ed'],
        ['Engineering', 'Eng'],
        ['Mechanical', 'Mech'],
        ['International', 'Int'],
        ['Biomaterials', 'Biomat'],
        ['Science', 'Sci'],
    ]
    
    for needle, replacement in pairs:
        name = name.replace(needle, replacement)
    return name

pub_name = []
for a in df['source']:
    try:
        a['title']
    except KeyError:
        continue
        
    pub_name.append(clean_journal(a['title']))
            
plot_statistic(pub_name)


# Topic Modeling
## Preprocessing

In [ ]:
# Load stopwords, bigrams, and stem rules
import nltk
nltk.download('punkt')
nltk.download('stopwords')

import spacy
spacy.cli.download("en_core_web_sm")
nlp = spacy.load("en_core_web_sm", disable=['ner'])

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

from sklearn.feature_extraction import text
stop_words = text.ENGLISH_STOP_WORDS.union(stop_words)

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

data_words = list(sent_to_words(texts))

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

In [ ]:
# Print common words
one_count = defaultdict(int)

for line in data_words_nostops:
    for a in line:
        one_count[a] += 1
        
print('Top words')
display(pd.DataFrame(
    [(w, one_count[w]) for w in top_k(one_count, 250)],
    columns=['word', 'count']))

In [ ]:
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Print common bigrams
two_count = defaultdict(int)

for line in data_words_bigrams:
    for a, b in zip(line, line[1:]):
        two_count[a, b] += 1
            
print('Top bigrams')
display(pd.DataFrame(
    [(w, two_count[w]) for w in top_k(two_count, 250)],
    columns=['bigram', 'count']))

In [ ]:
# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

report_sentences_lemma = [' '.join(w) for w in data_lemmatized]

large_string = ' '.join(report_sentences_lemma)

## Create Dictionary

In [ ]:
dic = gensim.corpora.Dictionary(data_lemmatized)
dic.filter_extremes(0, 0.5) # Remove 
dic.filter_extremes(5, 1)   # 
corpus = [dic.doc2bow(line) for line in data_lemmatized]

print('papers: {}'.format(len(corpus)))
print('dictionary size: {}'.format(len(dic)))

## Topic Modeling

In [ ]:
# Create frequency matrix
n, m = len(corpus), len(dic)
matrix = np.zeros((n, m))

for i, row in enumerate(corpus):
    for j, freq in row:
        matrix[i,j] = freq
        

In [ ]:
# Run TFIDF model
tfidf_model = sklearn.feature_extraction.text.TfidfTransformer()
tfidf_matrix = tfidf_model.fit_transform(matrix).toarray()

In [ ]:
nmf_model = sklearn.decomposition.NMF(
    n_components=num_topics,
    random_state=seed,
    tol=1e-9,
    max_iter=500,
    verbose=True)

# Train model
doc2topic = nmf_model.fit_transform(tfidf_matrix)
topic2token = nmf_model.components_

topic_norm = np.sum(topic2token, axis=1)
topic2token /= topic_norm[:,np.newaxis]
doc2topic *= topic_norm[np.newaxis,:]

doc_norm = np.sum(doc2topic, axis=1)
doc2topic /= doc_norm[:,np.newaxis]

In [ ]:
rows = []

for label, vec in zip(labels, topic2token):
    rows.append([label] + ['{} ({:.2})'.format(dic[i], vec[i]) for i in np.argsort(vec)[::-1][:10]])

# Each row is a topic, columns are words ordered by weight 
pd.DataFrame(rows)

In [ ]:
prepare_fig(2, 3)
for index in range(num_topics):
    mapping = dict()
    for i in np.argsort(topic2token[index])[::-1][:100]:
        if topic2token[index,i] > 0:
            mapping[dic[i]] = topic2token[index,i]
    
    def get_color(word, **kwargs):
        weight = kwargs['font_size'] / 60.0 * 0.6 + 0.4
        r, g, b = plt.get_cmap('Blues')(weight)[:3]
        return 'rgb(%s, %s, %s)' % (int(r * 255), int(g * 255), int(b * 255))
    
    wc = WordCloud(
        prefer_horizontal=True,
        max_font_size=75,
        #width=395,
        #height=250,
        scale=2,
        background_color='white', 
        color_func=get_color, 
        relative_scaling=0.5)
    wc.fit_words(mapping)
    
    print('Topic {} ({})'.format(index, labels[index]))
    plt.subplot(4, 3, index + 1)
    plt.imshow(wc.to_array(), interpolation='bilinear')
    plt.xticks([])
    plt.yticks([])
    
plt.subplots_adjust(left=0, right=1, top=1, bottom=0, wspace=0.1, hspace=0.1)
plt.show()


## Visualization

In [ ]:
import sklearn.manifold
import sklearn.metrics.pairwise
from sklearn.decomposition import TruncatedSVD

def draw_dot(p, t, zorder=0):
    color = plt.get_cmap('jet')(float(t) / num_topics)
    color = 0.8 * np.array(color)[:3]
    
    plt.scatter(
        p[0], 
        p[1],
        s=150,
        c=[color],
        marker='o',
        linewidth=0.5,
        zorder=zorder)
    
    plt.text(
        p[0], 
        p[1],
        labels[t],
        fontsize=6,
        color='1',
        va='center',
        ha='center',
        fontweight='bold',
        zorder=zorder + 1)

# Lower dimensionality of original frequency matrix to improve cosine distances for visualization
reduced_matrix = TruncatedSVD(
    n_components=10, 
    random_state=seed
).fit_transform(tfidf_matrix)

# Learn model, tune perplexity value according to complexity of data
model = sklearn.manifold.TSNE(
    verbose=True,
    metric='cosine',
    random_state=vis_seed,
    perplexity=5)
pos = model.fit_transform(reduced_matrix)

# Rotate visualization
theta = np.deg2rad(vis_angle + 60)
R = np.array([[np.cos(theta), np.sin(theta)], 
              [-np.sin(theta), np.cos(theta)]])
pos = np.dot(pos, R)

# Resize so xy-position is between 0.05 and 0.95
pos -= (np.amin(pos, axis=0) + np.amax(pos, axis=0)) / 2
pos /= np.amax(np.abs(pos))
pos = (pos * 0.5) + 0.5
pos = (pos * 0.9) + 0.05

prepare_fig(2, 4)
plt.xticks([])
plt.yticks([])
plt.xlim(0, 1)
plt.ylim(0, 1)
zorder = 0

# Draw dots
for i in np.random.permutation(len(doc2topic)):
    topic_id = np.argmax(doc2topic[i])
    draw_dot(pos[i], topic_id, zorder)
    zorder += 2

# Draw legend
for i in range(num_topics):    
    y = 0.985 - i * 0.02
    label = ', '.join(dic[w] for w in np.argsort(topic2token[i])[::-1][:3])

    draw_dot([0.015, y], i)
    plt.text(0.03, y, label, ha='left', va='center', fontsize=8, zorder=zorder)
    zorder += 1

plt.show()

In [ ]:
# Do an overall wordcloud
word_cloud = WordCloud(
    background_color="white",
    max_words=5000,
    width=1500,
    height=1000,
    stopwords=stop_words,
    contour_width=3,
    contour_color='steelblue'
)

# display our wordcloud across all records
plt.figure(figsize=(16,16))
word_cloud.generate(large_string)
word_cloud.to_file("all_words.png")
plt.imshow(word_cloud, interpolation='bilinear')
plt.axis("off")